In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from nltk.classify import ClassifierI
import numpy as np

## Importing 

In [2]:
nyt = pd.read_csv('../3. Sentiment Analysis preparation/sentiment_prepared.csv', index_col = 0)
nyt.head()

,articleWordCount,headline,snippet,tokens,label_headline
0,1324,G.O.P. Leadership Poised to Topple Obama’s Pi...,The most powerful and ambitious Republican-led...,"['leadership', 'poised', 'topple', 'obama', 'p...",Positive
1,2836,Fractured World Tested the Hope of a Young Pre...,A strategy that went from a “good war” to the ...,"['fractured', 'world', 'tested', 'hope', 'youn...",Positive
2,445,Little Troublemakers,Chuck Deodene puts us in a bubbly mood.,"['little', 'troublemakerschuck', 'deodene', 'p...",Neutral
3,864,"Angela Merkel, Russia’s Next Target","With a friend entering the White House, Vladim...","['angela', 'merkel', 'russia', 'next', 'target...",Negative
4,309,Boots for a Stranger on a Bus,Witnessing an act of generosity on a rainy day.,"['boots', 'stranger', 'buswitnessing', 'act', ...",Neutral


## New dataframe to eliminate neutral words

In [3]:
new_nyt = nyt[nyt.label_headline != 'Neutral'].reset_index(drop=True).copy()

In [4]:
new_nyt = new_nyt.replace(['Positive', 'Negative'], ['pos', 'neg'])     

In [5]:
new_nyt.head()

,articleWordCount,headline,snippet,tokens,label_headline
0,1324,G.O.P. Leadership Poised to Topple Obama’s Pi...,The most powerful and ambitious Republican-led...,"['leadership', 'poised', 'topple', 'obama', 'p...",pos
1,2836,Fractured World Tested the Hope of a Young Pre...,A strategy that went from a “good war” to the ...,"['fractured', 'world', 'tested', 'hope', 'youn...",pos
2,864,"Angela Merkel, Russia’s Next Target","With a friend entering the White House, Vladim...","['angela', 'merkel', 'russia', 'next', 'target...",neg
3,557,Sprint and Mr. Trump’s Fictional Jobs,"The emerging, and dangerous, new form of crony...","['sprint', 'mr', 'trump', 'fictional', 'jobsth...",neg
4,1109,"Fighting Diabetes, and Leading by Example","Eric L. Adams, the Brooklyn borough president,...","['fighting', 'diabetes', 'leading', 'exampleer...",pos


In [6]:
new_nyt.shape

(5279, 5)

## Find most common words

In [7]:
def clean_list_str(text):
    text = text.strip('[]').split(",")
    cleaned_text = [word.strip("' ") for word in text]
    return cleaned_text

In [8]:
all_words = {}

for token in new_nyt.tokens:
    cleaned_token = clean_list_str(token)
    for word in cleaned_token:
        all_words[word] = 1 if word not in all_words.keys() else all_words[word] + 1

In [9]:
all_words = nltk.FreqDist(all_words)
most_common_freq = all_words.most_common(1000)

In [10]:
most_common_words = [word for word, freq in most_common_freq]

## Find features

In [11]:
featuresets = []

for idx in range(new_nyt.shape[0]):
    # Read data
    tokens, label = new_nyt.loc[idx, ['tokens', 'label_headline']]
    # Transform str to list
    cleaned_tokens = clean_list_str(tokens)
    # Create token feature
    token_dict = {word: word in most_common_words for word in cleaned_tokens}
    # Save feature
    feature = (token_dict, label)
    featuresets.append(feature)

In [12]:
training_set = featuresets[:3500]
testing_set = featuresets[3500:]

In [13]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [14]:
print("Naive Bayes accuracy percent:", 
      (nltk.classify.accuracy(classifier, testing_set))*100)

Naive Bayes accuracy percent: 78.07757166947724


## Create a tuple with words, frequency, and positive or negative sentence